In [15]:
import numpy as np
import xarray as xr
import pandas as pd

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.units import units 
from metpy.interpolate import cross_section
from scipy import signal
from diagnosticSGfunctions import gkern as create_gaussian_kernel

import matplotlib

In [16]:
lon_start = 104.0
lon_end = 115.0
lat_start = 5.5
lat_end = 7.0

In [17]:
time = int(48)-12
gl_pe='/nobackup/earshar/borneo/case_20181021T1200Z_N768_v2/umglaa_pe{0:03d}.nc'.format(time)
gl_pc='/nobackup/earshar/borneo/case_20181021T1200Z_N768_v2/umglaa_pc{0:03d}.nc'.format(time)
gl_pb='/nobackup/earshar/borneo/case_20181021T1200Z_N768_v2/umglaa_pb{0:03d}.nc'.format(time)

input_data_pe = xr.open_dataset(gl_pe).metpy.assign_crs(grid_mapping_name='latitude_longitude', 
                                                        earth_radius=6371229.0)
input_data_pc = xr.open_dataset(gl_pc, drop_variables=['unspecified_5','unspecified_6',   # heating rate 
                                                       'unspecified_9','unspecified_10'])
input_data_pc = input_data_pc.metpy.assign_crs(grid_mapping_name='latitude_longitude', 
                                               earth_radius=6371229.0)
input_data_pb = xr.open_dataset(gl_pb).metpy.assign_crs(grid_mapping_name='latitude_longitude', 
                                                        earth_radius=6371229.0)

In [18]:
gdata_pe=input_data_pe.sel(longitude=slice(lon_start, lon_end), latitude=slice(lat_start, lat_end),
                                 longitude_1=slice(lon_start, lon_end), latitude_1=slice(lat_start, lat_end) )
gdata_pb=input_data_pb.sel(longitude=slice(lon_start, lon_end), latitude=slice(lat_start, lat_end) )
gdata_pc=input_data_pc.sel(longitude=slice(lon_start, lon_end), latitude=slice(lat_start, lat_end) )

In [19]:
heating_rate = 0
vars_list = ['unspecified', 'unspecified_1', 'unspecified_2', 'unspecified_3',
             'unspecified_4', 'unspecified_7','unspecified_8']
for var in vars_list:
    heating_rate = heating_rate + gdata_pc[var]

In [20]:
ht_coords = gdata_pe.u['hybrid_ht_1'].data.astype('int32')

u_gl = gdata_pe.u.interp(longitude_1=gdata_pe.v["longitude"],
                         hybrid_ht_1=ht_coords,
                         method="linear").assign_coords(height_levels=("hybrid_ht_1",
                        ht_coords)).swap_dims({"hybrid_ht_1":
                        "height_levels"})
v_gl = gdata_pe.v.interp(latitude_1=gdata_pe.u["latitude"],
                         hybrid_ht_1=ht_coords,
                         method="linear").assign_coords(height_levels=("hybrid_ht_1",
                        ht_coords)).swap_dims({"hybrid_ht_1":
                                               "height_levels"})
pv_gl = gdata_pe.field83.interp(hybrid_ht=ht_coords,
                                method="linear").assign_coords(height_levels=("hybrid_ht",
                                ht_coords)).swap_dims({"hybrid_ht":
                                                       "height_levels"})
w_gl  = gdata_pe.dz_dt.interp(hybrid_ht=ht_coords,
                              method="linear").assign_coords(height_levels=("hybrid_ht",
                            ht_coords)).swap_dims({"hybrid_ht":
                                                   "height_levels"})

q_gl  = gdata_pb.q.interp(hybrid_ht=ht_coords,
                          method="linear").assign_coords(height_levels=("hybrid_ht",
                            ht_coords)).swap_dims({"hybrid_ht":
                                                   "height_levels"})
th_gl = gdata_pb.theta.interp(hybrid_ht=ht_coords,
                              method="linear").assign_coords(height_levels=("hybrid_ht",
                            ht_coords)).swap_dims({"hybrid_ht":
                                                   "height_levels"})
rho_gl = gdata_pb.field27.interp(hybrid_ht_1=ht_coords,
                                 method="linear").assign_coords(height_levels=("hybrid_ht_1",
                                ht_coords)).swap_dims({"hybrid_ht_1":"height_levels"})
heating_rate = heating_rate.interp(hybrid_ht=ht_coords,
                                   method="linear").assign_coords(height_levels=("hybrid_ht",
                                ht_coords)).swap_dims({"hybrid_ht":"height_levels"})
exner_func = gdata_pb.field7.interp(hybrid_ht_1=ht_coords,
                                 method="linear").assign_coords(height_levels=("hybrid_ht_1",
                                ht_coords)).swap_dims({"hybrid_ht_1":"height_levels"})

In [21]:
u_gl = u_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
v_gl = v_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
w_gl = w_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))

th_gl = th_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
q_gl = q_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
rho_gl = rho_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))

vort_gl = mpcalc.vorticity(u_gl, v_gl, dx=None, dy=None)
abs_vort_gl = mpcalc.absolute_vorticity(u_gl, v_gl, dx=None, dy=None)

vort_gl = vort_gl.sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
abs_vort_gl = abs_vort_gl.sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
heating_rate = heating_rate.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end)) / 3600.
exner_func = exner_func.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))

### Calculate first term in Eq. 6 (absolute vorticity)

In [22]:
rho_upper = rho_gl.sel(height_levels=slice(8000,10000)).mean(dim=['height_levels'])
rho_lower = rho_gl.sel(height_levels=slice(2000,4000)).mean(dim=['height_levels'])
rho_ave = ( (rho_upper + rho_lower) / 2).mean() 

q_ave = q_gl.sel(height_levels=6000, method="nearest").mean()

th_upper = th_gl.sel(height_levels=slice(8000,10000)).mean(dim=['height_levels'])
th_lower = th_gl.sel(height_levels=slice(2000,4000)).mean(dim=['height_levels'])
d_theta  = (th_upper - th_lower).mean()
th_ref   = 300.0
dz       = 6000.0

# term_1 = (9.81 * d_theta / (rho_ave * q_ave * th_ref * dz) )
term_1 = (abs_vort_gl * d_theta / (rho_ave * dz) )

### Calculate second term in Eq. 6 (moist stability)

In [25]:
# calculate moist stability
th_dz = mpcalc.first_derivative(th_gl, axis=0)
th_dz = th_dz * (9.81 / th_ref)
moist_stability = q_gl * th_dz

moist_stability_upper = moist_stability.sel(height_levels=slice(8000,
                            10000)).mean(dim=['height_levels']).mean()
moist_stability_lower = moist_stability.sel(height_levels=slice(2000,
                            4000)).mean(dim=['height_levels']).mean()
moist_stability_interface = moist_stability.sel(height_levels=6000, method='nearest').mean()

term_2 = (moist_stability_upper - moist_stability_lower) / moist_stability_interface

moist_stratification_factor = term_1 * term_2 # Running code above --> factor = -3.3 (sensible?)

In [30]:
term_1

<xarray.DataArray (height_levels: 70, latitude: 10, longitude: 47)>
<Quantity([[[           nan            nan            nan ...            nan
              nan            nan]
  [7.42608627e-08 1.16878892e-07 9.48880570e-08 ... 7.96333428e-08
   1.06209391e-07            nan]
  [1.47072396e-07 1.06418006e-07 1.06012338e-07 ... 8.30659987e-08
   1.05005735e-07            nan]
  ...
  [5.74727983e-08 8.03194164e-08 8.52579853e-08 ... 1.51933365e-07
   1.47445189e-07            nan]
  [4.45353922e-08 7.03271708e-08 7.90002178e-08 ... 1.46160180e-07
   1.31187946e-07            nan]
  [           nan            nan            nan ...            nan
              nan            nan]]

 [[           nan            nan            nan ...            nan
              nan            nan]
  [7.42363953e-08 1.13447412e-07 9.36925656e-08 ... 7.93984472e-08
   1.07082861e-07            nan]
  [1.39908851e-07 1.03503399e-07 1.05986192e-07 ... 8.31430151e-08
   1.05894189e-07            nan]
  ...
  [5.22869243e-08 7.54674404e-08 7.98704477e-08 ... 1.53413628e-07
   1.48907173e-07            nan]
  [3.66365845e-08 6.44034351e-08 7.26241372e-08 ... 1.47710637e-07
   1.32098459e-07            nan]
  [           nan            nan            nan ...            nan
              nan            nan]]

 [[           nan            nan            nan ...            nan
              nan            nan]
  [7.49539448e-08 1.12406189e-07 9.34797579e-08 ... 7.94669892e-08
   1.07888106e-07            nan]
  [1.34768968e-07 1.02797258e-07 1.06300512e-07 ... 8.37245399e-08
   1.06977943e-07            nan]
  ...
  [5.13814153e-08 7.34117037e-08 7.71841229e-08 ... 1.53830101e-07
   1.49342032e-07            nan]
  [3.29962089e-08 6.16307329e-08 6.92597414e-08 ... 1.48288282e-07
   1.32334495e-07            nan]
  [           nan            nan            nan ...            nan
              nan            nan]]

 ...

 [[           nan            nan            nan ...            nan
              nan            nan]
  [1.10908324e-07 1.15384934e-07 1.39496397e-07 ... 8.75032872e-08
   7.55163464e-08            nan]
  [1.23976539e-07 1.31343114e-07 1.54138770e-07 ... 8.18159961e-08
   7.15301201e-08            nan]
  ...
  [9.57767593e-08 1.10420047e-07 1.26682429e-07 ... 1.74161920e-07
   1.81597084e-07            nan]
  [1.04101126e-07 1.05361112e-07 1.08315635e-07 ... 1.55202955e-07
   1.59004794e-07            nan]
  [           nan            nan            nan ...            nan
              nan            nan]]

 [[           nan            nan            nan ...            nan
              nan            nan]
  [1.42213484e-07 1.71887533e-07 1.98131462e-07 ... 2.53004732e-07
   2.47476808e-07            nan]
  [1.22148321e-07 1.66478749e-07 2.04485261e-07 ... 2.74421450e-07
   2.59972525e-07            nan]
  ...
  [1.76453439e-07 1.65590906e-07 1.66748145e-07 ... 2.86308896e-07
   2.69704744e-07            nan]
  [1.74743476e-07 1.62679821e-07 1.72754505e-07 ... 2.86591403e-07
   2.59235037e-07            nan]
  [           nan            nan            nan ...            nan
              nan            nan]]

 [[           nan            nan            nan ...            nan
              nan            nan]
  [9.84838907e-08 9.91623377e-08 1.00231843e-07 ... 2.14514424e-07
   2.25981453e-07            nan]
  [1.00368384e-07 1.04275577e-07 1.08537806e-07 ... 2.17577345e-07
   2.27563463e-07            nan]
  ...
  [1.09513119e-07 1.14113226e-07 1.25448712e-07 ... 2.13042476e-07
   2.11034839e-07            nan]
  [1.08121506e-07 1.16759466e-07 1.27873925e-07 ... 2.28956869e-07
   2.28531459e-07            nan]
  [           nan            nan            nan ...            nan
              nan            nan]]], '1 / second')>
Coordinates:
  * latitude       (latitude) float32 5.546875 5.703125 ... 6.796875 6.953125
    t              datetime64[ns] 2018-10-23T12:00:00
    metpy_crs      object Projection: latitude_longitude
    longitude_

In [26]:
moist_stratification_factor

<xarray.DataArray (height_levels: 70, latitude: 10, longitude: 47)>
<Quantity([[[            nan             nan             nan ...             nan
               nan             nan]
  [-1.84703337e-07 -2.90703885e-07 -2.36007772e-07 ... -1.98065893e-07
   -2.64166456e-07             nan]
  [-3.65801868e-07 -2.64685328e-07 -2.63676341e-07 ... -2.06603675e-07
   -2.61172695e-07             nan]
  ...
  [-1.42947674e-07 -1.99772312e-07 -2.12055634e-07 ... -3.77892182e-07
   -3.66729087e-07             nan]
  [-1.10769458e-07 -1.74919367e-07 -1.96491170e-07 ... -3.63532984e-07
   -3.26293697e-07             nan]
  [            nan             nan             nan ...             nan
               nan             nan]]

 [[            nan             nan             nan ...             nan
               nan             nan]
  [-1.84642481e-07 -2.82169028e-07 -2.33034318e-07 ... -1.97481656e-07
   -2.66338971e-07             nan]
  [-3.47984532e-07 -2.57436051e-07 -2.63611311e-07 ... -2.06795232e-07
   -2.63382478e-07             nan]
  ...
  [-1.30049248e-07 -1.87704365e-07 -1.98655626e-07 ... -3.81573927e-07
   -3.70365367e-07             nan]
  [-9.11233610e-08 -1.60185714e-07 -1.80632435e-07 ... -3.67389314e-07
   -3.28558346e-07             nan]
  [            nan             nan             nan ...             nan
               nan             nan]]

 [[            nan             nan             nan ...             nan
               nan             nan]
  [-1.86427186e-07 -2.79579276e-07 -2.32505017e-07 ... -1.97652135e-07
   -2.68341795e-07             nan]
  [-3.35200497e-07 -2.55679718e-07 -2.64393093e-07 ... -2.08241614e-07
   -2.66078016e-07             nan]
  ...
  [-1.27797045e-07 -1.82591289e-07 -1.91974137e-07 ... -3.82609789e-07
   -3.71446958e-07             nan]
  [-8.20689346e-08 -1.53289385e-07 -1.72264432e-07 ... -3.68826048e-07
   -3.29145418e-07             nan]
  [            nan             nan             nan ...             nan
               nan             nan]]

 ...

 [[            nan             nan             nan ...             nan
               nan             nan]
  [-2.75853750e-07 -2.86988078e-07 -3.46958667e-07 ... -2.17640201e-07
   -1.87826005e-07             nan]
  [-3.08357317e-07 -3.26679633e-07 -3.83377516e-07 ... -2.03494639e-07
   -1.77911370e-07             nan]
  ...
  [-2.38218173e-07 -2.74639296e-07 -3.15087470e-07 ... -4.33179560e-07
   -4.51672471e-07             nan]
  [-2.58922730e-07 -2.62056596e-07 -2.69405155e-07 ... -3.86024382e-07
   -3.95480403e-07             nan]
  [            nan             nan             nan ...             nan
               nan             nan]]

 [[            nan             nan             nan ...             nan
               nan             nan]
  [-3.53716669e-07 -4.27522649e-07 -4.92797157e-07 ... -6.29279226e-07
   -6.15530044e-07             nan]
  [-3.03810131e-07 -4.14069798e-07 -5.08600473e-07 ... -6.82547383e-07
   -6.46609682e-07             nan]
  ...
  [-4.38879078e-07 -4.11861534e-07 -4.14739846e-07 ... -7.12114114e-07
   -6.70815883e-07             nan]
  [-4.34626019e-07 -4.04621016e-07 -4.29679005e-07 ... -7.12816772e-07
   -6.44775383e-07             nan]
  [            nan             nan             nan ...             nan
               nan             nan]]

 [[            nan             nan             nan ...             nan
               nan             nan]
  [-2.44951413e-07 -2.46638862e-07 -2.49298960e-07 ... -5.33545241e-07
   -5.62066299e-07             nan]
  [-2.49638568e-07 -2.59356628e-07 -2.69957743e-07 ... -5.41163408e-07
   -5.66001114e-07             nan]
  ...
  [-2.72383564e-07 -2.83825057e-07 -3.12018941e-07 ... -5.29884177e-07
   -5.24890736e-07             nan]
  [-2.68922311e-07 -2.90406846e-07 -3.18050987e-07 ... -5.69466823e-07
   -5.68408735e-07             nan]
  [            nan             nan             nan ...             nan
               nan             nan]]], '1 / second')>


### Calculate third term in Eq. 6 (diabatic heating)

In [98]:
latitude = th_gl.latitude * (np.pi / 180.)
f0 = 1.4584E-4 * np.sin(latitude)
exner_func_6km = exner_func.sel(height_levels=6000, method="nearest").mean()
heating_rate_6km = (heating_rate.sel(height_levels=6000, method="nearest")) #* exner_func_6km)

# new method to calculate amplitude (smooth field, and calculate standard deviation)
kernel_2d = create_gaussian_kernel(3, 1.0)
heating_rate_smoothed = xr.DataArray(signal.convolve2d(heating_rate_6km, kernel_2d, boundary='symm', mode='same'))
heating_rate_mean = heating_rate_smoothed.mean()
heating_rate_std = (heating_rate_smoothed - heating_rate_mean).std()
heating_rate_amplitude = (heating_rate_6km.max() - heating_rate_6km.min()) / 2

rel_vort_6km = vort_gl.sel(height_levels=6000, method="nearest")
rel_vort_smoothed = xr.DataArray(signal.convolve2d(rel_vort_6km, kernel_2d, boundary='symm', mode='same'))
rel_vort_mean = rel_vort_smoothed.mean()
rel_vort_std = (rel_vort_smoothed - rel_vort_mean).std()
rel_vort_amplitude = (rel_vort_6km.max() - rel_vort_6km.min()) / 2

pv_wave_length = 8.0 * 111000 
pv_wave_number = (2 * np.pi) / pv_wave_length

term_3a = 1 / pv_wave_number
term_3b = heating_rate_std / d_theta
term_3c = f0 / rel_vort_std
term_3 = (term_3a * term_3b * term_3c).mean()

upper_wave_phase_speed = moist_stratification_factor * term_3

In [99]:
th_lower = th_gl.sel(height_levels=1400, method="nearest")

In [26]:
# fig, ax = plt.subplots(figsize=(12, 9))
# th_lower.plot()
# fig.tight_layout()
# plt.savefig('test.png')
# plt.close()

In [100]:
gdata_pb_theta = input_data_pb.sel(longitude=slice(104.0, 115.0), latitude=slice(5.5, 9.0) )

th_lower = gdata_pb_theta.theta.interp(hybrid_ht=ht_coords,
                                     method="linear").assign_coords(height_levels=("hybrid_ht",
                                    ht_coords)).swap_dims({"hybrid_ht":"height_levels"})

In [101]:
th_lower = th_lower.squeeze('t').sel(height_levels=1400, method="nearest")

In [102]:
th_ave_north = th_lower.sel(latitude=9.0, method="nearest").mean()
th_ave_south = th_lower.sel(latitude=5.5, method="nearest").mean()

In [103]:
th_dy = (th_ave_north - th_ave_south) / (3.5 * 111000)

In [104]:
## Lower-level wave (theta) ### 

theta_wave_length = 8.0 * 111000
theta_wave_number = (2 * np.pi) / theta_wave_length

theta_smoothed = xr.DataArray(signal.convolve2d(th_lower, kernel_2d, boundary='symm', mode='same'))
theta_mean = theta_smoothed.mean()
theta_std = (theta_smoothed - theta_mean).std()

v_lower = v_gl.sel(height_levels=1400, method="nearest")
v_smoothed = xr.DataArray(signal.convolve2d(v_lower, kernel_2d, boundary='symm', mode='same'))
v_mean = v_smoothed.mean()
v_std = (v_smoothed - v_mean).std()

base_state_gradient = - ( (1 / theta_wave_number) * (th_dy / d_theta) )
wave_amplitude = d_theta / theta_std
frequency = theta_wave_number * v_std
length_scale = 1 / theta_wave_number
lower_wave_phase_speed = base_state_gradient * wave_amplitude * frequency * length_scale

In [105]:
upper_wave_phase_speed, lower_wave_phase_speed

(<xarray.DataArray ()>
 <Quantity(-0.6371956549743868, '1 / kelvin')>
 Coordinates:
     t              datetime64[ns] 2018-10-22T12:00:00
     metpy_crs      object Projection: latitude_longitude
     hybrid_ht      int32 6196
     height_levels  int32 6196,
 <xarray.DataArray ()>
 array(-0.53889322)
 Coordinates:
     t              datetime64[ns] 2018-10-22T12:00:00
     metpy_crs      object Projection: latitude_longitude
     hybrid_ht      int32 1396
     height_levels  int32 1396)

In [26]:
# U1 = u_gl.sel(height_levels=1400, method="nearest").mean()
# U2 = u_gl.sel(height_levels=6000, method="nearest").mean()